In [1]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def dis_from_origin(self):
        return math.sqrt(self.x**2 + self.y**2)
    def __lt__(self, other):
        return self.dis_from_origin() < other.dis_from_origin()
    def __gt__(self, other):
        return self.dis_from_origin() > other.dis_from_origin()
    def __eq__(self, other):
        return self.dis_from_origin() == other.dis_from_origin()

In [2]:
import uuid
class Account:
    def __init__(self, initial_amount, minimum, interest_rate):
        if (initial_amount < minimum):
            raise ValueError("When creating this account, the initial amount must be >= {}.".format(minimum))
        self._id = str(uuid.uuid4())[:5]
        self._minimum       = minimum
        self._amount_held   = initial_amount
        self._min_ever_held = initial_amount
        self._interest_rate = interest_rate
        self._good_standing = True   
        self._is_active     = True

    def get_amount_held(self):
        return self._amount_held

    def get_minimum(self):
        return self._minimum

    def get_min_ever_held(self):
        return self._min_ever_held

    def get_interest_rate(self):
        return self._interest_rate

    def is_in_good_standing(self):
        return self._good_standing

    def is_active(self):
        return self._is_active;

    def withdraw(self, w_amount):
        if (w_amount > self._amount_held):
            raise ValueError("Cannot withdraw more than you have.")
        self._amount_held = self._amount_held - w_amount
        if (self._amount_held < self._minimum):
            self._good_standing = False;

        if (self._amount_held < self._min_ever_held):
            self._min_ever_held = self._amount_held

    def deposit(self, amount):
        self._amount_held += amount;
        if (self._amount_held >= self._minimum):
            self._good_standing = True

    def close_account(self):
        self._is_active = False;
        self._amount_held += self._min_ever_held * self._interest_rate
        return self._amount_held

In [3]:
import uuid
class SavingAccount(Account):
    def __init__(self, initial_amount, max_num_withdrawals=1,
                 minimum=1000, interest_rate=0.10,
                 bonus_contribution=0.15):
        super().__init__(initial_amount, minimum, interest_rate)
        self._num_withdrawals = 0
        self._max_num_withdrawals = max_num_withdrawals
        self._bonus_contribution = bonus_contribution
        self._min_balance_ever = initial_amount
    def get_num_withdrawals(self):
        return self._num_withdrawals
    def withdraw(self, w_amount):
        if self._num_withdrawals >= self._max_num_withdrawals:
            raise ValueError("Savings accounts allow only {} withdrawals.".format(self._max_num_withdrawals))
        self._num_withdrawals += 1
        if self.balance - w_amount < self._min_balance_ever: 
            self._min_balance_ever = self.balance - w_amount
        super().withdraw(w_amount)
    def add_bonus(self):
        bonus_amount = (self._bonus_contribution * self._min_balance_ever) + 100
        self.deposit(bonus_amount) 
    def close_account(self):
        self.add_bonus()
        return super().close_account() 

In [4]:
import unittest

class TestPointMethods(unittest.TestCase):
    def test_dis_from_origin(self):
        point = Point(3, 4)
        self.assertEqual(point.dis_from_origin(), 5)

    def test_comparator_lt(self):
        point1 = Point(3, 4)
        point2 = Point(1, 2)
        self.assertFalse(point1 < point2)

    def test_comparator_gt(self):
        point1 = Point(3, 4)
        point2 = Point(1, 2)
        self.assertTrue(point1 > point2)

    def test_comparator_eq(self):
        point1 = Point(3, 4)
        point2 = Point(0, 5)
        self.assertTrue(point1 == point2)

In [5]:
class TestSavingAccountMethods(unittest.TestCase):
    def setUp(self):
        # Setup a SavingAccount object before each test
        self.account = SavingAccount(1000, max_num_withdrawals=2, minimum=500, interest_rate=0.05, bonus_contribution=0.1)

    def test_withdraw(self):
        # Test withdrawal functionality and withdrawal count increment
        self.account.withdraw(100)
        self.assertEqual(self.account.get_num_withdrawals(), 1)
        self.account.withdraw(100)
        self.assertEqual(self.account.get_num_withdrawals(), 2)
        with self.assertRaises(ValueError):
            # Test exceeding maximum withdrawals
            self.account.withdraw(100)

    def test_add_bonus(self):
        # Assuming initial balance is the lowest ever held
        self.account.add_bonus()
        # Bonus should be 10% of 1000 plus 100 = 200
        self.assertEqual(self.account.get_amount_held(), 1200)

    def test_close_account(self):
        # Test closing the account with the initial amount and bonus added
        initial_balance = self.account.get_amount_held()
        closed_amount = self.account.close_account()
        # Assuming no withdrawals or deposits, closed amount includes bonus and interest
        expected_closed_amount = initial_balance * (1 + self.account._interest_rate) + (self.account._bonus_contribution * initial_balance) + 100
        self.assertEqual(closed_amount, expected_closed_amount)
        self.assertFalse(self.account.is_active())
